In [1]:
from transformers import LlamaForCausalLM, LlamaTokenizer, AutoTokenizer, AutoModelForCausalLM
import os
import numpy as np
import torch

In [2]:
base_model = "meta-llama/Llama-2-7b-chat-hf"
# llama_model = "meta-llama/Meta-Llama-3-8B-Instruct"

device_map = 'auto'
max_memory_mapping = {0: "10GiB", 1: "10GiB"}

In [3]:
tokenizer = LlamaTokenizer.from_pretrained(base_model,
                                           padding_side = "left",
                                           add_eos_token = True,
                                           add_bos_token = True,                                        
                                           token = os.environ.get("HUGGINGFACE_ACCESS_TOKEN"),
)
tokenizer.add_special_tokens({"pad_token":tokenizer.eos_token})

0

In [4]:
model = LlamaForCausalLM.from_pretrained(
        base_model,
        load_in_8bit=True,
        torch_dtype=torch.float16,
        device_map=device_map,
        max_memory = max_memory_mapping,
        token = os.environ.get("HUGGINGFACE_ACCESS_TOKEN"),
    ).eval()
# model.resize_token_embeddings(len(tokenizer),pad_to_multiple_of=8)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
prompt = ['Prediction: Yes']
inputs = tokenizer(prompt, return_tensors="pt", padding = True)
print(inputs)

{'input_ids': tensor([[    1, 21099,  2463, 29901,  3869,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1]])}


In [8]:
prompt = ['Prediction: No']
inputs = tokenizer(prompt, return_tensors="pt", padding = True)
print(inputs)

{'input_ids': tensor([[    1, 21099,  2463, 29901,  1939,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1]])}


In [6]:
prompt = ['[/INST]']
inputs = tokenizer(prompt, return_tensors="pt", padding = True)
print(inputs)

{'input_ids': tensor([[    1,   518, 29914, 25580, 29962,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1]])}


In [6]:
tokenizer.vocab_size

32000

In [7]:
import json
train_data_path = './final_data/beauty/test.json'
with open(train_data_path, 'r') as f:
    train_data = json.load(f)
print(len(train_data))

604


In [8]:
def create_data(d):
    return  f"""[INST]{d['instruction']}{d['input']}[/INST]"""

In [9]:
prompt = [
    create_data(train_data[0]),
    create_data(train_data[1])
]
print(prompt)
inputs = tokenizer(prompt, return_tensors="pt", padding = True)
print(inputs)

['[INST]You are an expert beauty product recommender. Given a user\'s profile and list of recent liked and disliked items and their price, brand, and description, identify whether the user will like or dislike the target item by providing reasoning for the same. Please follow further instruction below. User Profile - \n\nThe user seems to prefer hypoallergenic fragrances and natural ingredients. They liked Fresh Eau De Parfum EDP - Fig Apricot, which is a hypoallergenic fragrance, indicating their preference for pleasant scents that don\'t cause allergies. They also liked Monoi - Monoi Pitate Jasmine, a jasmine-scented coconut oil, suggesting a preference for natural ingredients that are beneficial for dry skin and hair. The user may not prefer overpowering or artificial scents, as indicated by their dislike for KMRN Fresh Line, which may not offer a wide range of scents and may have a non-natural jasmine scent.\n        User purchased the following items in the given order. List of re

In [10]:
from transformers import GenerationConfig

generation_config = GenerationConfig(
            do_sample = True,
            temperature=0.01,
            top_p=0.75,
            top_k=40,
            num_beams=1,
        )

In [11]:
generate_ids = model.generate(**inputs,
                generation_config=generation_config,
                return_dict_in_generate=True,
                max_new_tokens=64,
                output_scores=True)
print(generate_ids)

/home/grads/m/mbismay/miniconda3/envs/mixtral/lib/python3.9/site-packages/transformers/generation/utils.py:1413: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


GenerateDecoderOnlyOutput(sequences=tensor([[    1,   518, 25580,  ...,   278,  1404, 29915],
        [    2,     2,     2,  ..., 29995, 26783,   880]]), scores=(tensor([[-inf, -inf, -inf,  ..., -inf, -inf, -inf],
        [-inf, -inf, -inf,  ..., -inf, -inf, -inf]]), tensor([[-inf, -inf, -inf,  ..., -inf, -inf, -inf],
        [-inf, -inf, -inf,  ..., -inf, -inf, -inf]]), tensor([[-inf, -inf, -inf,  ..., -inf, -inf, -inf],
        [-inf, -inf, -inf,  ..., -inf, -inf, -inf]]), tensor([[-inf, -inf, -inf,  ..., -inf, -inf, -inf],
        [-inf, -inf, -inf,  ..., -inf, -inf, -inf]]), tensor([[-inf, -inf, -inf,  ..., -inf, -inf, -inf],
        [-inf, -inf, -inf,  ..., -inf, -inf, -inf]]), tensor([[-inf, -inf, -inf,  ..., -inf, -inf, -inf],
        [-inf, -inf, -inf,  ..., -inf, -inf, -inf]]), tensor([[-inf, -inf, -inf,  ..., -inf, -inf, -inf],
        [-inf, -inf, -inf,  ..., -inf, -inf, -inf]]), tensor([[-inf, -inf, -inf,  ..., -inf, -inf, -inf],
        [-inf, -inf, -inf,  ..., -inf, -inf,

In [12]:
scores = generate_ids.scores
print(len(scores), scores[0].shape)

64 torch.Size([2, 32000])


In [13]:
tokenizer.batch_decode(generate_ids, skip_special_tokens=False, clean_up_tokenization_spaces=True)

ValueError: invalid literal for int() with base 10: 's'

In [14]:
messages = [
    {"role": "system", "content": train_data[0]['instruction']},
    {"role": "user", "content": train_data[0]['input']},
]

In [15]:
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)
print(input_ids)

tensor([[    1, 29961, 25580,  ..., 29914, 25580, 29962]], device='cuda:0')


In [16]:
tokenizer.decode(input_ids[0])

'<s>[INST] <<SYS>>\nYou are an expert beauty product recommender. Given a user\'s profile and list of recent liked and disliked items and their price, brand, and description, identify whether the user will like or dislike the target item by providing reasoning for the same. Please follow further instruction below. \n<</SYS>>\n\nUser Profile - \n\nThe user seems to prefer hypoallergenic fragrances and natural ingredients. They liked Fresh Eau De Parfum EDP - Fig Apricot, which is a hypoallergenic fragrance, indicating their preference for pleasant scents that don\'t cause allergies. They also liked Monoi - Monoi Pitate Jasmine, a jasmine-scented coconut oil, suggesting a preference for natural ingredients that are beneficial for dry skin and hair. The user may not prefer overpowering or artificial scents, as indicated by their dislike for KMRN Fresh Line, which may not offer a wide range of scents and may have a non-natural jasmine scent.\n        User purchased the following items in t

In [17]:
response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))

NameError: name 'outputs' is not defined

{'input_ids': tensor([[   1,  584, 3869,    2]]), 'attention_mask': tensor([[1, 1, 1, 1]])}


In [19]:
tokenizer.batch_decode([29901, 3869], skip_special_tokens=True, clean_up_tokenization_spaces=False)

[':', 'Yes']

In [44]:
l = [[1,2,1,3,1,3,4,45,1,3], [1,2,1,3,1,3,4,45,1,3]]
l1 = [[n if n!=1 else 0 for n in label] for label in l]
l1, l

([[0, 2, 0, 3, 0, 3, 4, 45, 0, 3], [0, 2, 0, 3, 0, 3, 4, 45, 0, 3]],
 [[1, 2, 1, 3, 1, 3, 4, 45, 1, 3], [1, 2, 1, 3, 1, 3, 4, 45, 1, 3]])

In [20]:
from transformers import AutoTokenizer, LlamaForCausalLM

prompt = "Hey, are you conscious? Can you talk to me?"
inputs = tokenizer(prompt, return_tensors="pt")

# Generate
generate_ids = model.generate(inputs.input_ids, max_length=30)
tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

'Hey, are you conscious? Can you talk to me?\nЉ�?�?�?�?�?�?�?'

### Mistral 7B Instruct v0.2

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import os
import torch
# device = "cuda" # the device to load the model onto
device_map = 'auto'
max_memory_mapping = {0: "23GiB", 1: "23GiB"}
model_id = "mistralai/Mistral-7B-Instruct-v0.2"

model = AutoModelForCausalLM.from_pretrained(model_id,
                                             token = os.environ.get("HUGGINGFACE_ACCESS_TOKEN"))
tokenizer = AutoTokenizer.from_pretrained(model_id,
                                        load_in_8bit=True,
                                        torch_dtype=torch.float16,
                                        device_map=device_map,
                                        max_memory = max_memory_mapping,
                                        token = os.environ.get("HUGGINGFACE_ACCESS_TOKEN"),)

messages = [
    {"role": "user", "content": create_data(train_data[0])}
]

encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")

model_inputs = encodeds

generated_ids = model.generate(model_inputs, max_new_tokens=1000, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] You are an expert movie critic and recommender. Given the user profile and list of recent liked and disliked movies, identify whether the user will like the target movie or not by providing reasoning for the same. Please follow further instruction below.
User Profile - 
            The user has a diverse taste in movies, with a preference for thought-provoking and emotionally engaging films. They enjoy dramas that explore complex themes such as mental illness, racial injustice, and the meaning of life. The user is drawn to movies that challenge their perspective and provoke deep thought, often favoring those that tackle serious and mature subjects.
            
            In addition to dramas, the user also has a soft spot for classic Disney films, as evidenced by their fondness for Cinderella. They appreciate the enchanting storytelling and timeless charm of these animated tales.
            
            The user's taste in movies also extends to comedies, as seen in thei

In [6]:
print(decoded[0].split("[/INST]")[1])


 Based on the user profile and movie watching history, the user has a diverse taste in movies, with a preference for thought-provoking and emotionally engaging films. They enjoy dramas that explore complex themes and appreciate enchanting storytelling in animated films, particularly those with a deep message or exploration of identity. The user tends to avoid horror movies, violence, and graphic content.

The user's recent movie preferences demonstrate a liking for animated films with strong narratives and complex themes, such as "Snow White and the Seven Dwarfs," "Beauty and the Beast," and "The Hunchback of Notre Dame." They also enjoy comedies, as shown by their appreciation for "Airplane!" and "Wallace & Gromit."

Given the user's history and the user profile, it is likely that they will enjoy "Mulan" due to its strong narrative, themes of empowerment, and its visual storytelling in an animated format. Additionally, the user's appreciation for deeper storytelling in animated films,

In [1]:
import operator
import time
import gc
gc.collect()

start = time.time()
def rindex(lst, value):
    return len(lst) - operator.indexOf(reversed(lst), value) - 1
lst = [3]*10000000000
lst.extend([2,5,7,3,8,9])
val = 3
print(time.time()-start, rindex(lst, val))

16.17045783996582 10000000003


In [ ]:
def find_r_index(lst, val):
    n = len(lst)
    

In [11]:
from math import log

start = time.time()
x = log(10000000000)
print(time.time() - start, x)

8.225440979003906e-05 23.025850929940457


In [ ]:
100 - 0.00024
1000 - 0.00011
100000 - 0.0043
1000000 - 0.0033
100000000 - 0.181
10000000000 - 16.06

### Testing Similarity

In [1]:
from sklearn.metrics.pairwise import cosine_similarity

In [9]:
l1 = [[1,1,1,1,1,1,1,1,1,1,1,1,1,2,3,4,7385,6221]]
l2 = [[1,1,1,1,1,1,1,1,1,1,1,1,22,1123,3,10,9,12]]
cosine_similarity(l1, l2)

array([[0.01322412]])

In [3]:
import os
path = "./lora-llama2-chat/sample64_epoch5_valsample1600"
CONFIG_NAME = 'adapter_config.json'

In [4]:
if os.path.isfile(os.path.join(path, CONFIG_NAME)):
    config_file = os.path.join(path, CONFIG_NAME)

In [8]:
print(os.path.join(path, CONFIG_NAME))
os.path.isfile(os.path.join(path, CONFIG_NAME))

./lora-llama2-chat/sample64_epoch5_valsample1600/adapter_config.json


False

In [5]:
import torch
labels_index = torch.tensor([[0,2097],
                [1,2097],
                [2,2097],
                [3,2097]])
print(labels_index[:, 0])
print(labels_index[:, 1])

tensor([0, 1, 2, 3])
tensor([2097, 2097, 2097, 2097])


In [1]:
x = [1,2,3,4]
y = [5,6]
x+y

[1, 2, 3, 4, 5, 6]

In [10]:
golds = ['Yes', 'No', 'Hat No']
assert all(elem.split(' ')[-1] in ['Yes', 'No'] for elem in golds), "Yes/No is not present"

In [1]:
from scipy.special import softmax

softmax([1.2744e-09, 5.5356e-09])

array([0.5, 0.5])

In [4]:
softmax([1.0000, 1.5535e-06])

array([0.73105827, 0.26894173])

### BERTScore

In [2]:
from evaluate import load
bertscore = load("bertscore")

In [3]:
predictions = ["The user's behavior from the given list of movies suggests that they enjoy heartwarming, visually stunning, and story-driven films with themes of love, second chances, and justice. They appreciate movies that feature talking animals, fantasy elements, and adventure. However, they tend to avoid psychological thrillers and slasher films.\n\nThe user will likely enjoy Superstar because it is a heartwarming comedy about an awkward high school girl who dreams of becoming a famous celebrity. The movie features talking animals, fantasy elements, and adventure, all of which align with the user's preferences. Additionally, the movie's themes of love, second chances, and justice are likely to resonate with the user. Overall, Superstar is a perfect fit for the user's movie preferences.\nPrediction: Yes",
               "The user's movie watching history suggests that they enjoy thought-provoking narratives with intricate storytelling and complex themes. They appreciate historical dramas, psychological thrillers, and musicals. However, they tend to avoid modern sci-fi and action-packed genres.\n\nThe user will likely dislike The Unbearable Lightness of Being because it is a drama set in 1968 Prague, which aligns with their preference for historical dramas. However, the movie's focus on existential themes and complex relationships may not resonate with the user's taste for psychological thrillers and action-packed genres. The user's preference for classic films and avoidance of modern sci-fi and action-packed genres suggests that they may find The Unbearable Lightness of Being's focus on existential themes and complex relationships to be too cerebral and not engaging enough for their taste.\nPrediction: No"]
references = ["Summary of user behavior:\nThe user enjoys a wide range of genres, including children's, fantasy, adventure, comedy, romance, and drama. They have a preference for heartwarming, visually stunning movies with strong storytelling, often exploring themes of love, second chances, and justice. They appreciate movies that are imaginative, such as those featuring talking animals or magical elements. However, they tend to avoid psychological thrillers and slasher films with dark and unsettling themes.\n\nReasoning for liking Superstar:\nSuperstar aligns with the user's preferences by being a heartwarming comedy about an awkward high school girl who dreams of becoming a famous celebrity. The movie features a strong storyline with themes of love and second chances, which the user enjoys. Additionally, the user has shown a fondness for imaginative and visually stunning films, and Superstar, with its comedic and lighthearted approach, fits this description. The user's dislike for psychological thrillers and slasher films further supports the likelihood of them enjoying Superstar.\nPrediction: Yes",
              "Summary of user behavior:\nThe user prefers classic, thought-provoking films with exceptional performances and intricate storytelling. They enjoy psychological thrillers, historical dramas, musicals, and period dramas. The user dislikes modern sci-fi and action-packed genres, such as 'The Matrix' and 'The World Is Not Enough.' They appreciate suspenseful narratives, complex themes, and powerful performances.\n\nReasoning for liking Unbearable Lightness of Being, The:\nThe user will likely enjoy 'Unbearable Lightness of Being, The' because it is a thought-provoking and sensual drama set in 1968 Prague, exploring complex relationships and existential dilemmas. This aligns with their preference for classic, introspective films with intricate storytelling and exceptional performances. The historical setting and exploration of political turmoil also resonate with their interest in historical dramas like Schindler\'s List and The Godfather series.\nPrediction: Yes"]

In [4]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
results = bertscore.compute(predictions=predictions,
                            references=references,
                            model_type = "microsoft/deberta-xlarge-mnli")

/home/grads/m/mbismay/miniconda3/envs/mixtral/lib/python3.9/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [5]:
results

{'precision': [0.7760025262832642, 0.6887653470039368],
 'recall': [0.7266504764556885, 0.6837465763092041],
 'f1': [0.7505160570144653, 0.6862468123435974],
 'hashcode': 'microsoft/deberta-xlarge-mnli_L40_no-idf_version=0.3.12(hug_trans=4.37.2)'}